# Богданов Александр Иванович, Б05-003

## Анализ модели LSTM

In [300]:
from tqdm.notebook import tqdm
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

from prettytable import PrettyTable

from nerus import load_nerus

In [301]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Вспомогательные функции

In [302]:
def get_sent_tags(docs, size=10000):
    list_of_sent = []
    list_of_tags = []
    for doc in tqdm(docs):
        for sent in doc.sents:
            list_of_sent_toks = []
            list_of_sent_tags = []
            for tok in sent.tokens:
                list_of_sent_toks.append(tok.text)
                list_of_sent_tags.append(tok.pos)
        list_of_sent.append(list_of_sent_toks)
        list_of_tags.append(list_of_sent_tags)
        if len(list_of_sent) > size:
            break
    return list_of_sent, list_of_tags

In [303]:
def pos_dict(list_of_tags, test_size=100):
    pos2idx = {'<PAD>' : 0}
    idx2pos = ['<PAD>']

    for tags in list_of_tags[:-test_size]:
        for word in tags:
            if word not in pos2idx:
                pos2idx[word] = len(idx2pos)
                idx2pos.append(word)
    return pos2idx, idx2pos

In [304]:
def word_dict(list_of_tags, test_size=100):
    word2idx = {'<PAD>': 0, '<UNK>': 1}
    idx2word = ['<PAD>', '<UNK>']
    
    for sent in list_of_sent[:-test_size]:
        for word in sent:
            if word not in word2idx:
                word2idx[word] = len(idx2word)
                idx2word.append(word)
                
    return word2idx, idx2word

In [305]:
class NerusDataset(Dataset):
    def __init__(self, list_of_sent, list_of_tags, word2idx, pos2idx, train=True, test_size=100):
        self.X = []
        self.y = []
        
        if train:
            for sent in list_of_sent[:-test_size]:
                data = []
                for word in sent:
                    data.append(word2idx.get(word, 0))
                self.X.append(data)
            
            for tags in list_of_tags[:-test_size]:
                data = []
                for word in tags:
                    data.append(pos2idx.get(word, 0))
                self.y.append(data)
        else:
            for sent in list_of_sent[-test_size:]:
                data = []
                for word in sent:
                    data.append(word2idx.get(word, 0))
                self.X.append(data)
                
            for tags in list_of_tags[-test_size:]:
                data = []
                for word in tags:
                    data.append(pos2idx.get(word, 0))
                self.y.append(data)
            
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.Tensor(self.X[idx]), torch.Tensor(self.y[idx])

In [306]:
def collate_fn(data):
    X, Y = [], []
    for x, y in data:
        X.append(x)
        Y.append(y)
    x_batch = torch.zeros((len(X), max(list(map(len, X)))), dtype=torch.long)
    y_batch = torch.zeros((len(Y), max(list(map(len, Y)))), dtype=torch.long)
    
    for i, sent in enumerate(X):
        x_batch[i, :len(sent)] = sent

    for i, sent in enumerate(Y):
        y_batch[i, :len(sent)] = sent
        
    return x_batch, y_batch

In [327]:
def check(batch_size, dataset, model, loss_function, idx2word, idx2pos):
    
    model.eval()
    
    batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, collate_fn=collate_fn)
            
    test_acc = 0
    test_loss = 0
    count = 0
    for it, (x_batch, y_batch) in enumerate(batch_generator):
        x_batch = x_batch.to(model.device)
        y_batch = y_batch.to(model.device)
                
        mask = (y_batch != 0)
        count += mask.sum()
                
        output = model(x_batch)

        test_loss += loss_function(output.transpose(1,2), y_batch).cpu().item()*len(x_batch)
        test_acc += (torch.argmax(output, dim=-1).cpu() == y_batch)[mask].sum().item()
            
    test_loss /= len(dataset)
    test_acc /= count

    print(f'loss: {test_loss}, acc: {test_acc}')
    
    dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    x, y = next(iter(dataloader))
    x = x.to(device)
    y = y.to(device)

    outputs = model(x)
    
    one_x = x[0].cpu().numpy()
    one_y = y[0].cpu().numpy()
    one_output = outputs[0].argmax(dim=-1).cpu().numpy()

    words = [idx2word[idx] for idx in one_x]
    true_tags = [idx2pos[idx] for idx in one_y]
    pred_tags = [idx2pos[idx] for idx in one_output]

    table = PrettyTable(["Word", "True tag", "Predicted tag"])
    table.align["Word"], table.align["True tag"], table.align["Predicted tag"] = "l", "l", "l"

    for word, true_tag, pred_tag in zip(words, true_tags, pred_tags):
        if word != idx2word[word2idx['<PAD>']]:
            table.add_row([word, true_tag, pred_tag])

    print(table)

    return test_loss, test_acc

In [308]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    model.train()
    model.zero_grad()
    
    output = model(x_batch.to(device))
    
    loss = loss_function(output.transpose(1, 2), y_batch.to(device))
    loss.backward()

    optimizer.step()
    return loss.cpu().item()

In [309]:
def train_epoch(train_generator, model, loss_function, optimizer, callback = None):
    epoch_loss = 0
    total = 0
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(model, batch_of_x.to(device), batch_of_y.to(device), optimizer, loss_function)
        train_generator.set_postfix({'train batch loss': batch_loss})
        
        if callback is not None:
            callback(model, batch_loss)
            
        epoch_loss += batch_loss*len(batch_of_x)
        total += len(batch_of_x)
    
    return epoch_loss/total

In [310]:
def trainer(count_of_epoch, 
            batch_size, 
            dataset,
            model, 
            loss_function,
            optimizer,
            lr = 0.001,
            callback = None):

    optima = optimizer(model.parameters(), lr=lr)
    
    iterations = tqdm(range(count_of_epoch), desc='epoch')
    iterations.set_postfix({'train epoch loss': np.nan})
    for it in iterations:
        batch_generator = tqdm(
            torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn), 
            leave=False, total=len(dataset)//batch_size+(len(dataset)%batch_size> 0))
        
        epoch_loss = train_epoch(
                    train_generator=batch_generator, 
                    model=model, 
                    loss_function=loss_function, 
                    optimizer=optima, 
                    callback=callback)
        
        iterations.set_postfix({'train epoch loss': epoch_loss})

In [311]:
class callback():
    def __init__(self, writer, dataset, loss_function, delimeter = 300, batch_size=64):
        self.step = 0
        self.writer = writer
        self.delimeter = delimeter
        self.loss_function = loss_function
        self.batch_size = batch_size

        self.dataset = dataset

    def forward(self, model, loss):
        model.eval()
        self.step += 1
        self.writer.add_scalar('LOSS/train', loss, self.step)
        
        if self.step % self.delimeter == 0:
            
            batch_generator = torch.utils.data.DataLoader(dataset=self.dataset, 
                                                          batch_size=self.batch_size,
                                                          collate_fn=collate_fn)
            
            test_acc = 0
            test_loss = 0
            count = 0
            for it, (x_batch, y_batch) in enumerate(batch_generator):
                x_batch = x_batch.to(model.device)
                y_batch = y_batch.to(model.device)
                
                mask = (y_batch != 0)
                count += mask.sum()
                
                output = model(x_batch)

                test_loss += self.loss_function(output.transpose(1,2), y_batch).cpu().item()*len(x_batch)

                test_acc += (torch.argmax(output, dim=-1).cpu() == y_batch)[mask].sum().item()
            
            test_loss /= len(self.dataset)
            test_acc /= count

            print(f"\t step={self.step}, train_loss={loss}, val_loss={test_loss}, val_acc={test_acc}")
            
            self.writer.add_scalar('LOSS/test', test_loss, self.step)
            self.writer.add_scalar('ACC/test', test_acc, self.step)
          
    def __call__(self, model, loss):
        return self.forward(model, loss)

## Модель

In [312]:
class LSTM(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device
        
    def __init__(self,
                 vocab_dim,
                 output_dim = 18,
                 emb_dim = 10, 
                 hidden_dim = 10,
                 num_layers = 3,
                 dropout = 0,
                 batch_norm = False,
                 bidirectional = False):
        super(LSTM, self).__init__()
        
        self.embedding = torch.nn.Embedding(vocab_dim, emb_dim, padding_idx=0)
        self.encoder = torch.nn.LSTM(emb_dim, hidden_dim, num_layers, bidirectional = bidirectional, batch_first=True)
        if batchnorm:
            self.batch_norm = torch.nn.BatchNorm1d(hidden_dim)
        else:
            self.batch_norm = None
        self.dropout = torch.nn.Dropout(p=dropout)
        self.linear = torch.nn.Linear(hidden_dim, output_dim)
        
    def forward(self, input):
        out = self.embedding(input)
        out, _ = self.encoder(out)
        if self.batch_norm is not None:
            out = self.batch_norm(out.transpose(1, 2)).transpose(1, 2)
        out = self.dropout(out)
        out = self.linear(out)
        return out

## Подключим tensorboard

In [313]:
%load_ext tensorboard
%tensorboard --logdir tensorboard_2/

## Обработка данных

In [314]:
docs = load_nerus("nerus_lenta.conllu.gz")

In [315]:
list_of_sent, list_of_tags = get_sent_tags(docs, 10000)

0it [00:00, ?it/s]

In [316]:
pos2idx, idx2pos = pos_dict(list_of_tags)

In [317]:
word2idx, idx2word = word_dict(list_of_sent)

In [318]:
train_data = NerusDataset(list_of_sent, list_of_tags, word2idx, pos2idx, train=True)
test_data = NerusDataset(list_of_sent, list_of_tags, word2idx, pos2idx, train=False)

## Обучение

In [319]:
loss_function = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam

In [320]:
dim_list = [10, 20, 30]
num_layers_list = [3, 5, 7, 9]
dropout_list = [0, 0.3, 0.5]
batch_norm_list = [False, True]
len_dict_list = [50000, 100000, 150000]

In [321]:
for dim in dim_list:
    print(f'dim = {dim}')
    
    model = LSTM(vocab_dim=len(word2idx), emb_dim=dim, hidden_dim=dim, output_dim=len(pos2idx))
    model.to(device)
    
    writer = SummaryWriter(log_dir=f'tensorboard_2/dim_{dim}')
    call = callback(writer, test_data, loss_function)
    
    check(64, test_data, model, loss_function, idx2word, idx2pos)
    trainer(count_of_epoch=10, 
            batch_size=64, 
            dataset=train_data,
            model=model, 
            loss_function=loss_function,
            optimizer = optimizer,
            callback=call)
    check(64, test_data, model, loss_function, idx2word, idx2pos)

dim = 10
loss: 2.943786954879761, acc: 0.022016221657395363
+------------+----------+---------------+
| Word       | True tag | Predicted tag |
+------------+----------+---------------+
| В          | ADP      | ADV           |
| нем        | PRON     | ADV           |
| они        | PRON     | ADV           |
| опровергли | VERB     | ADV           |
| информацию | NOUN     | ADV           |
| о          | ADP      | ADV           |
| связи      | NOUN     | ADV           |
| с          | ADP      | ADV           |
| российской | ADJ      | ADV           |
| разведкой  | NOUN     | ADV           |
| и          | CCONJ    | ADV           |
| утверждали | VERB     | ADV           |
| ,          | PUNCT    | ADV           |
| что        | SCONJ    | ADV           |
| с          | ADP      | ADV           |
| визитом    | NOUN     | ADV           |
| и          | CCONJ    | ADV           |
| собирались | VERB     | ADV           |
| посмотреть | VERB     | ADV           |
| на         | A

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=300, train_loss=2.168388605117798, val_loss=2.2419653129577637, val_acc=0.2665121555328369


...

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1200, train_loss=0.6944470405578613, val_loss=0.7773849034309387, val_acc=0.740440309047699


  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1500, train_loss=0.544668436050415, val_loss=0.638272910118103, val_acc=0.7972189784049988
loss: 0.6303366827964783, acc: 0.8070683479309082
+-------------+----------+---------------+
| Word        | True tag | Predicted tag |
+-------------+----------+---------------+
| Группа      | NOUN     | NOUN          |
| вошла       | VERB     | VERB          |
| в           | ADP      | ADP           |
| Зал         | PROPN    | ADJ           |
| славы       | NOUN     | NOUN          |
| рок-н-ролла | NOUN     | NOUN          |
| в           | ADP      | ADP           |
| 1996        | ADJ      | ADJ           |
| году        | NOUN     | NOUN          |
| .           | PUNCT    | PUNCT         |
+-------------+----------+---------------+


Как мы видим - лучше всего использовать размер слоя 30.

In [322]:
for num_layers in num_layers_list:
    print(f'num_layers = {num_layers}')
    
    model = LSTM(vocab_dim=len(word2idx), num_layers=num_layers)
    model.to(device)
    
    writer = SummaryWriter(log_dir=f'tensorboard_2/num_layers_{num_layers}')
    call = callback(writer, test_data, loss_function)
    
    check(64, test_data, model, loss_function, idx2word, idx2pos)
    trainer(count_of_epoch=10, 
            batch_size=64, 
            dataset=train_data,
            model=model, 
            loss_function=loss_function,
            optimizer = optimizer,
            callback=call)
    check(64, test_data, model, loss_function, idx2word, idx2pos)

num_layers = 3
loss: 2.7688625144958494, acc: 0.26998841762542725
+--------------------+----------+---------------+
| Word               | True tag | Predicted tag |
+--------------------+----------+---------------+
| Он                 | PRON     | NOUN          |
| призвал            | VERB     | NOUN          |
| бороться           | VERB     | NOUN          |
| с                  | ADP      | NOUN          |
| мздоимством        | NOUN     | NOUN          |
| ,                  | PUNCT    | NOUN          |
| нанося             | VERB     | NOUN          |
| удары              | NOUN     | NOUN          |
| «                  | PUNCT    | NOUN          |
| и                  | CCONJ    | NOUN          |
| по                 | ADP      | NOUN          |
| тигру              | NOUN     | NOUN          |
| ,                  | PUNCT    | NOUN          |
| и                  | CCONJ    | NOUN          |
| по                 | ADP      | NOUN          |
| мухе               | NOUN     | 

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=300, train_loss=2.179889440536499, val_loss=2.2437978076934812, val_acc=0.26998841762542725


...

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1200, train_loss=2.1517250537872314, val_loss=2.2059787845611574, val_acc=0.26998841762542725


  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1500, train_loss=2.1995115280151367, val_loss=2.1966159248352053, val_acc=0.2885283827781677
loss: 2.195254373550415, acc: 0.2885283827781677
+------------+----------+---------------+
| Word       | True tag | Predicted tag |
+------------+----------+---------------+
| По         | ADP      | ADP           |
| их         | DET      | NOUN          |
| заявлениям | NOUN     | NOUN          |
| ,          | PUNCT    | NOUN          |
| твит       | NOUN     | NOUN          |
| Маска      | PROPN    | NOUN          |
| принес     | VERB     | NOUN          |
| им         | PRON     | NOUN          |
| убыток     | NOUN     | NOUN          |
| .          | PUNCT    | NOUN          |
+------------+----------+---------------+


Как мы видим - лучше всего использовать 3 слоя.

In [323]:
for dropout in dropout_list:
    print(f'dropout = {dropout}')
    
    model = LSTM(vocab_dim=len(word2idx), dropout=dropout)
    model.to(device)
    
    writer = SummaryWriter(log_dir=f'tensorboard_2/dropout_{dropout}')
    call = callback(writer, test_data, loss_function)
    
    check(64, test_data, model, loss_function, idx2word, idx2pos)
    trainer(count_of_epoch=10, 
            batch_size=64, 
            dataset=train_data,
            model=model, 
            loss_function=loss_function,
            optimizer = optimizer,
            callback=call)
    check(64, test_data, model, loss_function, idx2word, idx2pos)

dropout = 0
loss: 2.934212760925293, acc: 0.0
+-----------+----------+---------------+
| Word      | True tag | Predicted tag |
+-----------+----------+---------------+
| Одни      | DET      | INTJ          |
| пользу    | NOUN     | INTJ          |
| ,         | PUNCT    | INTJ          |
| другие    | ADJ      | INTJ          |
| указывают | VERB     | INTJ          |
| на        | ADP      | INTJ          |
| вред      | NOUN     | INTJ          |
| .         | PUNCT    | INTJ          |
+-----------+----------+---------------+


epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=300, train_loss=2.216475486755371, val_loss=2.240416326522827, val_acc=0.2665121555328369


...

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1200, train_loss=1.850610613822937, val_loss=1.71686119556427, val_acc=0.3933951258659363


  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1500, train_loss=1.8044638633728027, val_loss=1.615329966545105, val_acc=0.39455386996269226
loss: 1.5980723190307617, acc: 0.39455386996269226
+---------------+----------+---------------+
| Word          | True tag | Predicted tag |
+---------------+----------+---------------+
| Во            | ADP      | ADP           |
| ФСИН          | PROPN    | NOUN          |
| пояснили      | VERB     | NOUN          |
| ,             | PUNCT    | NOUN          |
| что           | SCONJ    | ADP           |
| это           | PRON     | ADP           |
| было          | AUX      | ADP           |
| сделано       | VERB     | NOUN          |
| по            | ADP      | ADP           |
| правительства | NOUN     | NOUN          |
| .             | PUNCT    | NOUN          |
+---------------+----------+---------------+


Как мы видим - лучше всего использовать dropout c 0.5.

In [324]:
for batch_norm in batch_norm_list:
    print(f'batchnorm = {batch_norm}')
    
    model = LSTM(vocab_dim=len(word2idx), batch_norm=batch_norm)
    model.to(device)
    
    writer = SummaryWriter(log_dir=f'tensorboard_2/batch_norm_{batch_norm}')
    call = callback(writer, test_data, loss_function)
    
    check(64, test_data, model, loss_function, idx2word, idx2pos)
    trainer(count_of_epoch=10, 
            batch_size=64, 
            dataset=train_data,
            model=model, 
            loss_function=loss_function,
            optimizer = optimizer,
            callback=call)
    check(64, test_data, model, loss_function, idx2word, idx2pos)

batch_norm = False
loss: 2.8620294666290285, acc: 0.16164541244506836
+---------------+----------+---------------+
| Word          | True tag | Predicted tag |
+---------------+----------+---------------+
| В             | ADP      | PUNCT         |
| результате    | NOUN     | PUNCT         |
| землетрясения | NOUN     | PUNCT         |
| были          | AUX      | PUNCT         |
| разрушены     | VERB     | PUNCT         |
| тысячи        | NOUN     | PUNCT         |
| домов         | NOUN     | PUNCT         |
| ,             | PUNCT    | PUNCT         |
| а             | CCONJ    | PUNCT         |
| также         | ADV      | PUNCT         |
| отели         | NOUN     | PUNCT         |
| ,             | PUNCT    | PUNCT         |
| больницы      | NOUN     | PUNCT         |
| и             | CCONJ    | PUNCT         |
| торговые      | ADJ      | PUNCT         |
| центры        | NOUN     | PUNCT         |
| .             | PUNCT    | PUNCT         |
+---------------+----------+--

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=300, train_loss=2.1636157035827637, val_loss=2.2413926887512208, val_acc=0.26998841762542725


...

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1200, train_loss=0.8655945658683777, val_loss=0.883663227558136, val_acc=0.6969872117042542


  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1500, train_loss=0.7061347365379333, val_loss=0.7359313678741455, val_acc=0.7497103214263916
loss: 0.7095850706100464, acc: 0.7555040121078491
+-----------------+----------+---------------+
| Word            | True tag | Predicted tag |
+-----------------+----------+---------------+
| В               | ADP      | ADP           |
| Кремле          | PROPN    | NOUN          |
| соответствующие | ADJ      | NOUN          |
| инициативы      | NOUN     | NOUN          |
| комментировать  | VERB     | ADJ           |
| не              | PART     | PART          |
| стали           | VERB     | VERB          |
| .               | PUNCT    | PUNCT         |
+-----------------+----------+---------------+


BatchNorm надо использовать.

In [326]:
for len_dict in len_dict_list:
    print(f'len_dict = {len_dict}')
    
    list_of_sent, list_of_tags = get_sent_tags(docs, len_dict)
    pos2idx, idx2pos = pos_dict(list_of_tags)
    word2idx, idx2word = word_dict(list_of_sent)
    
    train_data = NerusDataset(list_of_sent, list_of_tags, word2idx, pos2idx, train=True)
    test_data = NerusDataset(list_of_sent, list_of_tags, word2idx, pos2idx, train=False)
    
    model = LSTM(vocab_dim=len(word2idx))
    model.to(device)
    
    writer = SummaryWriter(log_dir=f'tensorboard_2/len_dict_{len_dict}')
    call = callback(writer, test_data, loss_function)
    
    check(64, test_data, model, loss_function, idx2word, idx2pos)
    trainer(count_of_epoch=10, 
            batch_size=64, 
            dataset=train_data,
            model=model, 
            loss_function=loss_function,
            optimizer = optimizer,
            callback=call)
    check(64, test_data, model, loss_function, idx2word, idx2pos)

len_dict = 50000


0it [00:00, ?it/s]

loss: 2.751299066543579, acc: 0.11717922240495682
+-------------+----------+---------------+
| Word        | True tag | Predicted tag |
+-------------+----------+---------------+
| Первые      | ADJ      | ADP           |
| пять        | NUM      | ADP           |
| миллиардов  | NOUN     | ADP           |
| были        | AUX      | ADP           |
| перечислены | VERB     | ADP           |
| в           | ADP      | ADP           |
| марте       | NOUN     | ADP           |
| 2015        | ADJ      | ADP           |
| года        | NOUN     | ADP           |
| ,           | PUNCT    | ADP           |
| еще         | ADV      | ADP           |
| 1,7         | NUM      | ADP           |
| миллиарда   | NOUN     | ADP           |
| —           | PUNCT    | ADP           |
| в           | ADP      | ADP           |
| августе     | NOUN     | ADP           |
| того        | DET      | ADP           |
| же          | PART     | ADP           |
| года        | NOUN     | ADP           |
| . 

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/780 [00:00<?, ?it/s]

...

	 step=16500, train_loss=0.18736568093299866, val_loss=0.2144860190153122, val_acc=0.9416204690933228
	 step=16800, train_loss=0.1712111085653305, val_loss=0.21607169270515442, val_acc=0.9391041398048401
	 step=17100, train_loss=0.18433357775211334, val_loss=0.20838802874088289, val_acc=0.9401106834411621
	 step=17400, train_loss=0.13598328828811646, val_loss=0.20756876826286316, val_acc=0.9426270127296448
	 step=17700, train_loss=0.1456078439950943, val_loss=0.20535694122314452, val_acc=0.9431303143501282
	 step=18000, train_loss=0.18662507832050323, val_loss=0.2000361567735672, val_acc=0.9436335563659668
	 step=18300, train_loss=0.17808927595615387, val_loss=0.19702111780643464, val_acc=0.9451434016227722
	 step=18600, train_loss=0.17649084329605103, val_loss=0.19480936706066132, val_acc=0.9426270127296448


  0%|          | 0/2343 [00:00<?, ?it/s]

	 step=18900, train_loss=0.16719451546669006, val_loss=0.19709946751594543, val_acc=0.9451434016227722
	 step=19200, train_loss=0.12367787957191467, val_loss=0.1954007351398468, val_acc=0.9451434016227722
	 step=19500, train_loss=0.13029004633426666, val_loss=0.1919304597377777, val_acc=0.9451434016227722
	 step=19800, train_loss=0.15529634058475494, val_loss=0.1858449923992157, val_acc=0.9471564888954163
	 step=20100, train_loss=0.10111507773399353, val_loss=0.18420552790164949, val_acc=0.9456466436386108
	 step=20400, train_loss=0.1354769915342331, val_loss=0.18298677265644073, val_acc=0.9456466436386108
	 step=20700, train_loss=0.11335772275924683, val_loss=0.18566490650177003, val_acc=0.9436335563659668
	 step=21000, train_loss=0.14709541201591492, val_loss=0.1803484845161438, val_acc=0.9441368579864502


  0%|          | 0/2343 [00:00<?, ?it/s]

	 step=21300, train_loss=0.15767942368984222, val_loss=0.19300721287727357, val_acc=0.9461499452590942
	 step=21600, train_loss=0.15034255385398865, val_loss=0.18673058211803437, val_acc=0.9431303143501282
	 step=21900, train_loss=0.1252322494983673, val_loss=0.18280743360519408, val_acc=0.9471564888954163
	 step=22200, train_loss=0.10544056445360184, val_loss=0.17784545242786406, val_acc=0.9466531872749329
	 step=22500, train_loss=0.11438729614019394, val_loss=0.17644255697727204, val_acc=0.9471564888954163
	 step=22800, train_loss=0.10694093257188797, val_loss=0.17822689235210418, val_acc=0.9461499452590942
	 step=23100, train_loss=0.10398882627487183, val_loss=0.18195416688919067, val_acc=0.9446401000022888
	 step=23400, train_loss=0.1125495582818985, val_loss=0.17482637882232666, val_acc=0.9486662745475769
loss: 0.16918574810028075, acc: 0.9496728181838989
+------------------------+----------+---------------+
| Word                   | True tag | Predicted tag |
+------------------

Чем больше словарь, тем лучше, но и на маленьком словаре сеть хорошо обучилась.

Были проведены эксперименты над моделью LSTM.

Как мы видим из графиков: размер слоев надо поставить побольше, главное не уйти в переобучение; количество слоев надо поставить поменьше, так как очень долго учится, dropout надо использовать с параметром 0.5; batchnorm себя хорошо показал; размер словаря надо брать как можно больше, так как увеличиваем размер выборки.